<a href="https://colab.research.google.com/github/sdalal1/Visual-Odometry/blob/main/Visual_Odometry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import progressbar

In [ ]:
file_path = '../dataset/sequences/00/image_0/'
left_images = os.listdir(file_path) # list of strings with names of images
print(len(left_images))

In [ ]:
plt.figure(figsize=(12,4))
plt.imshow(cv2.imread(file_path + left_images[0], 0), cmap='gray')
plt.show()

In [ ]:
file_path = '../dataset/sequences/00/'
velodyne_files = os.listdir(file_path + 'velodyne/')
pointcloud = np.fromfile(file_path + 'velodyne/' + velodyne_files[1], dtype=np.float32)

In [ ]:
len(pointcloud)

In [ ]:
pointcloud = pointcloud.reshape((-1, 4)) # 4 columns for x, y, z, intensity

In [ ]:
# %matplotlib notebook

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')

xs = pointcloud[:, 0][::10]
ys = pointcloud[:, 1][::10]
zs = pointcloud[:, 2][::10]

ax.set_box_aspect([np.ptp(xs), np.ptp(ys), np.ptp(zs)])
ax.grid(False)
ax.axis('off')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

ax.view_init(elev=40, azim=180)

ax.scatter(xs, ys, zs, s=0.1)
plt.show()

In [ ]:
calib = pd.read_csv('../dataset/sequences/00/calib.txt', delimiter=" ", header=None, index_col=0)
print(calib)
Tr = np.array(calib.iloc[4]).reshape((3, 4))
print(Tr.round(4))

In [ ]:
class Dataset_Handler():

    def __init__(self, sequence, lidar=True, progress_bar=True, low_memory=True):

        self.lidar = lidar
        self.low_memory = low_memory

        self.seq_dir = f"../dataset/sequences/{sequence}/"
        self.poses_dir = f"../dataset/poses/{sequence}.txt"
        poses = pd.read_csv(self.poses_dir, delimiter=" ", header=None)

        self.left_image_files = sorted(os.listdir(self.seq_dir + 'image_0'))
        self.right_image_files = sorted(os.listdir(self.seq_dir + 'image_1'))
        self.velodyne_files = sorted(os.listdir(self.seq_dir + 'velodyne'))
        self.num_frames = len(self.left_image_files)
        self.lidar_path = self.seq_dir + 'velodyne/'

        self.gt = np.zeros((self.num_frames, 3, 4))

        for i in range(len(poses)):
            self.gt[i] = np.array(poses.iloc[i]).reshape((3, 4))

        calib = pd.read_csv(self.seq_dir + 'calib.txt', delimiter=" ", header=None, index_col=0)
        self.P0 = np.array(calib.loc['P0:']).reshape((3, 4))
        self.P1 = np.array(calib.loc['P1:']).reshape((3, 4))
        self.P2 = np.array(calib.loc['P2:']).reshape((3, 4))
        self.P3 = np.array(calib.loc['P3:']).reshape((3, 4))
        if self.lidar:
            self.Tr = np.array(calib.loc['Tr:']).reshape((3, 4))

        if low_memory:
            self.reset_frames()
            self.first_image_left = cv2.imread(self.seq_dir + 'image_0/' + 
                                               self.left_image_files[0], 0)
            self.first_image_right = cv2.imread(self.seq_dir + 'image_1/' + 
                                               self.right_image_files[0], 0)
            self.second_image_left = cv2.imread(self.seq_dir + 'image_0/' + 
                                               self.left_image_files[1], 0)
            if lidar:
                self.first_pointcloud = np.fromfile(self.lidar_path + self.velodyne_files[0],
                                                    dtype=np.float32,
                                                    count=-1).reshape((-1, 4))
            self.imheight = self.first_image_left.shape[0]
            self.imwidth = self.first_image_left.shape[1]
        else:
            self.images_left = []
            self.images_right = []
            self.pointclouds = []
            if progress_bar:
                bar = progressbar.ProgressBar(maxval=self.num_frames)
                bar.start()
            for i, name_left in enumerate(self.left_image_files):
                name_right = self.right_image_files[i]
                self.images_left.append(cv2.imread(self.seq_dir + 'image_0/' + name_left))
                self.images_right.append(cv2.imread(self.seq_dir + 'image_1/' + name_right))
                if lidar:
                    pointcloud = np.fromfile(self.lidar_path + self.velodyne_files[i],
                                             dtype=np.float32, count=-1).reshape((-1, 4))
                    self.pointclouds.append(pointcloud)
                if progress_bar:
                    bar.update(i+1)
                self.imheight = self.images_left[0].shape[0]
                self.imwidth = self.images_right[0].shape[1]

    def reset_frames(self):
        self.images_left = (cv2.imread(self.seq_dir + 'image_0/' + name_left, 0)
                            for name_left in self.left_image_files)
        self.images_right= (cv2.imread(self.seq_dir + 'image_1/' + name_right, 0)
                            for name_right in self.right_image_files)
        if self.lidar:
            self.pointclouds = (np.fromfile(self.lidar_path + velodyne_file,
                                           dtype=np.float32,
                                           count=-1).reshape((-1, 4))
                               for velodyne_file in self.velodyne_files)
        pass
        

In [ ]:
handler = Dataset_Handler("00", lidar=True, low_memory=True)

In [ ]:
plt.imshow(handler.first_image_left, cmap='gray')
plt.show()
plt.imshow(handler.first_image_right, cmap='gray')
plt.show()

In [ ]:
def compute_left_disparity_map(img_left, img_right, matcher='bm', rgb=False, verbose=True):
    # the rgb argument can be taken out, we will not be using it.
    sad_window = 6
    num_disparities = sad_window*16
    block_size = 11
    matcher_name = matcher
    
    if matcher_name == 'bm':
        matcher = cv2.StereoBM_create(numDisparities=num_disparities,
                                      blockSize=block_size
                                     )
        
    elif matcher_name == 'sgbm':
        matcher = cv2.StereoSGBM_create(numDisparities=num_disparities,
                                        minDisparity=0,
                                        blockSize=block_size,
                                        P1 = 8 * 1 * block_size ** 2,
                                        P2 = 32 * 1 * block_size ** 2,
                                        mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY
                                       )
    if rgb:
        img_left = cv2.cvtColor(img_left, cv2.COLOR_BGR2GRAY)
        img_right = cv2.cvtColor(img_right, cv2.COLOR_BGR2GRAY)
    start = datetime.datetime.now()
    disp_left = matcher.compute(img_left, img_right).astype(np.float32)/16
    end = datetime.datetime.now()
    if verbose:
        print(f'Time to compute disparity map using Stereo{matcher_name.upper()}:', end-start)
    
    return disp_left
    

In [ ]:
disp = compute_left_disparity_map(handler.first_image_left,
                                  handler.first_image_right,
                                  matcher='sgbm',
                                  verbose=False)
plt.figure(figsize=(12, 4))
plt.imshow(disp);
plt.show()

In [ ]:
k, r, t, _, _, _, _ = cv2.decomposeProjectionMatrix(handler.P1)
print(k)
print(r)
print((t / t[3]).round())

In [ ]:
def decompose_projection_matrix(P):
    k, r, t, _, _, _, _ = cv2.decomposeProjectionMatrix(P)
    t = (t / t[3])[:3]
    return k, r, t

In [ ]:
def calc_depth_map(disp_left, k_left, t_left, t_right, rectified=True):
    # he explains what's going on with the rectified term and what it means
    # in video 3 ~48:00, it's not what you think
    if rectified:
        b = t_right[0] - t_left[0]
    else:
        b = t_left[0] - t_right[0]

    f = k_left[0, 0]
    
    disp_left[disp_left == 0] = 0.1
    disp_left[disp_left == -1] = 0.1

    depth_map = np.ones(disp_left.shape)
    depth_map = f * b / disp_left

    return depth_map

In [ ]:
k_left, r_left, t_left = decompose_projection_matrix(handler.P0)
k_right, r_right, t_right = decompose_projection_matrix(handler.P1)

In [ ]:
depth = calc_depth_map(disp, k_left, t_left, t_right)
plt.figure(figsize=(12, 4))
depth[depth == depth.max()] = 650
plt.imshow(depth)
plt.show()

In [ ]:
plt.hist(depth.flatten())
plt.show()

In [ ]:
mask = np.zeros(depth.shape, dtype=np.uint8)
ymax = depth.shape[0]
xmax = depth.shape[1]
cv2.rectangle(mask, (96, 0), (xmax, ymax), (255), thickness=-1)
plt.imshow(mask);
plt.show()

In [ ]:
def stereo_2_depth(img_left, img_right, P0, P1, matchers='bm', rgb=False, verbose=True,
                   rectified=True):
    # Compute disparity map
    disp = compute_left_disparity_map(img_left,
                                      img_right,
                                      matcher=matchers,
                                      rgb=rgb,
                                      verbose=verbose)
    # decompose projection matrix
    k_left, r_left, t_left = decompose_projection_matrix(P0)
    k_right, r_right, t_right = decompose_projection_matrix(P1)

    # calculate depth map for left camera
    depth = calc_depth_map(disp, k_left, t_left, t_right, rectified=rectified)

    return depth

In [ ]:
stereo_depth = stereo_2_depth(handler.first_image_left,
                       handler.first_image_right,
                       handler.P0,
                       handler.P1,
                       matchers='sgbm',
                       verbose=True)
plt.imshow(stereo_depth)
plt.show()

In [ ]:
pcloud = handler.first_pointcloud
print(pcloud.shape)
trimmed_pcloud = pcloud[pcloud[:, 0] > 0]
print(trimmed_pcloud.shape)

In [ ]:
hom_pcloud = np.hstack([trimmed_pcloud[:, :3], np.ones(trimmed_pcloud.shape[0]).reshape((-1, 1))])
cam_xyz = handler.Tr.dot(trimmed_pcloud.T)
cam_xyz /= cam_xyz[2]
cam_xyz = np.vstack([cam_xyz, np.ones(cam_xyz.shape[1])])
projection = handler.P0.dot(cam_xyz)
projection[:,:5].T

In [ ]:
pixel_coords = np.round(projection.T, 0).astype('int')
pixel_coords[:5]

In [ ]:
def pointcloud2image(pointcloud, imheight, imwidth, Tr, P0):
    
    pointcloud = pointcloud[pointcloud[:, 0] > 0]
    reflectance = pointcloud[:, 3]
    # make pointcloud homogenous (X, Y, Z, 1)
    pointcloud = np.hstack([pointcloud[:, :3], np.ones(pointcloud.shape[0]).reshape((-1, 1))])

    # Transform points into 3D coordinate frame of camera
    cam_xyz = Tr @ pointcloud.T

    # clip off points with negative z values
    cam_xyz = cam_xyz[:, cam_xyz[2] > 0]

    depth = cam_xyz[2].copy()

    cam_xyz /= cam_xyz[2]
    cam_xyz = np.vstack([cam_xyz, np.ones(cam_xyz.shape[1])])
    projection = P0 @ cam_xyz
    pixel_coordinates = np.round(projection.T,0)[:,:2].astype(int)

    # filter out points that are outside of the image
    indices = np.where((pixel_coordinates[:, 0] < imwidth)
                       & (pixel_coordinates[:, 0] >= 0)
                       & (pixel_coordinates[:, 1] < imheight)
                       & (pixel_coordinates[:, 1] >= 0))

    pixel_coordinates = pixel_coordinates[indices]
    depth = depth[indices]
    reflectance = reflectance[indices]

    render = np.zeros((imheight, imwidth))
    for j, (u,v) in enumerate(pixel_coordinates):
        # if u >= imwidth or u < 0:
        #     continue
        # if v >= imheight or v < 0:
        #     continue
        render[v,u] = depth[j]

    # render[render == 0.0] = 3861.45

    return render

In [ ]:
render = pointcloud2image(handler.first_pointcloud, handler.imheight, handler.imwidth, handler.Tr, handler.P0)

In [ ]:
plt.figure(figsize=(13,5))
plt.imshow(render)
plt.show()

In [ ]:
for i, d in enumerate(stereo_depth[200:, :].flatten()):
    if render[200:,:].flatten()[i] == 0:
        continue
    print("stereo depth: ", d, "lidar depth: ", render[200:,:].flatten()[i])
    if i > 1000:
        break

In [ ]:
handler.reset_frames()

pcloud_frames = (pointcloud2image(next(handler.pointclouds),
                                  handler.imheight,
                                  handler.imwidth,
                                  handler.Tr,
                                  handler.P0)
                for _ in range(handler.num_frames))

# poses = (gt for gt in handler.gt)

In [ ]:
handler.reset_frames()
poses = (gt for gt in handler.gt)

In [ ]:
# display the images and the pointclouds in cool ways for fun

xs = []
ys = []
zs = []

compute_times = []
fig = plt.figure(figsize=(4, 4))
ax = fig.add_subplot(projection='3d')
ax.view_init(elev=-20, azim=270)
ax.plot(handler.gt[:, 0, 3], handler.gt[:, 1, 3], handler.gt[:, 2, 3], c='k')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')


stereo_l = handler.images_left
stereo_r = handler.images_right

for i in range(handler.num_frames // 50):
    img_l = next(stereo_l)
    img_r = next(stereo_r)
    start = datetime.datetime.now()
    disp = compute_left_disparity_map(img_l, img_r, matcher='sgbm')
    disp /= disp.max()
    # disp = 1 - disp
    disp = (disp*255).astype('uint8')
    disp = cv2.applyColorMap(disp, cv2.COLORMAP_RAINBOW)
    pcloud = next(pcloud_frames)
    pcloud /= pcloud.max()
    pcloud = (pcloud*255).astype('uint8')

    gt = next(poses)
    xs.append(gt[0, 3])
    ys.append(gt[1, 3])
    zs.append(gt[2, 3])
    plt.plot(xs, ys, zs, c='chartreuse')
    plt.pause(0.000000000000000001)
    cv2.imshow('camera', img_l)
    cv2.imshow('disparity', disp)
    cv2.imshow('lidar', pcloud)
    cv2.waitKey(1)

    end = datetime.datetime.now()
    compute_times.append(end-start)

plt.close()
cv2.destroyAllWindows()

In [ ]:
def extract_features(image, detector='sift', mask=None):
    if detector == 'sift':
        det = cv2.SIFT_create()
    elif detector == 'orb':
        det = cv2.ORB_create()
    
    kp, des = det.detectAndCompute(image, mask) # key points, descriptors

    return kp, des

In [ ]:
def match_features(des1, des2, matching='BF', detector='sift', sort=False, k=2):

    if matching == 'BF':
        if detector == 'sift':
            matcher = cv2.BFMatcher_create(cv2.NORM_L2, crossCheck=False)
        elif detector == 'orb':
            matcher = cv2.BFMatcher_create(cv2.NORM_HAMMING2, crossCheck=False)
    elif matching == 'FLANN':
        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm=FLANN_INDEX_KDTREE)
        search_params = dict(checks=50)
        matcher = cv2.FlannBasedMatcher(index_params, search_params)

    matches = matcher.knnMatch(des1, des2, k=k)

    if sort:
        matches = sorted(matches, key=lambda x: x[0].distance)

    return matches

In [ ]:
def visualize_matches(image1, kp1, image2, kp2, match):
    image_matches = cv2.drawMatches(image1, kp1, image2, kp2, match, None, flags=2)
    plt.figure(figsize=(16,6), dpi=100)
    plt.imshow(image_matches)
    plt.show()

In [ ]:
def filter_matches_distance(matches, dist_threshold):
    filtered_matches = []
    # m, n are the two nearest neighbors for each match
    for m, n in matches:
        if m.distance <= dist_threshold * n.distance:
            filtered_matches.append(m)

    return filtered_matches

In [ ]:
# using sift

image_left = handler.first_image_left
image_right = handler.first_image_right
image_plus1 = handler.second_image_left

start = datetime.datetime.now()
kp0, des0 = extract_features(image_left, 'sift', mask)
kp1, des1 = extract_features(image_plus1, 'sift', mask)
matches = match_features(des0, des1, matching='BF', detector='sift', sort=False, k=2)
print('number of matches before filtering:', len(matches))
matches = filter_matches_distance(matches, 0.3)
print('number of matches after filtering:', len(matches))
end = datetime.datetime.now()
print('time to compute matches:', end-start)
visualize_matches(image_left, kp0, image_right, kp1, matches)

In [ ]:
# using orb

image_left = handler.first_image_left
image_right = handler.first_image_right
image_plus1 = handler.second_image_left

start = datetime.datetime.now()
kp0, des0 = extract_features(image_left, 'orb', mask)
kp1, des1 = extract_features(image_plus1, 'orb', mask)
matches = match_features(des0, des1, matching='BF', detector='orb', sort=False, k=2)
print('number of matches before filtering:', len(matches))
matches = filter_matches_distance(matches, 0.3)
print('number of matches after filtering:', len(matches))
end = datetime.datetime.now()
print('time to compute matches:', end-start)
visualize_matches(image_left, kp0, image_right, kp1, matches)

In [ ]:
def estimate_motion(matches, kp1, kp2, k, depth1, max_depth=3000):
    
    rmat = np.eye(3)
    tvec = np.zeros((3, 1))

    image1_points = np.float32([kp1[m.queryIdx].pt for m in matches])
    image2_points = np.float32([kp2[m.trainIdx].pt for m in matches])

    cx = k[0, 2]
    cy = k[1, 2]
    fx = k[0, 0]
    fy = k[1, 1]

    object_points = np.zeros((0, 3))
    delete = []

    for i, (u, v) in enumerate(image1_points):
        z = depth1[int(round(v)), int(round(u))]

        if z > max_depth:
            delete.append(i)
            continue
        
        # we need to subtract cx and cy to move the origin back into the center
        # of the image

        # we're trying to go back to xy coordinates in meters now

        # pretty sure this equation is in his notes?
        x = z * (u - cx) / fx
        y = z * (v - cy) / fy

        # neural network with monocular depth estimation? that sounds like a
        # portfolio post to me right there. He mentions this in video 5
        # around 26:50
        
        object_points = np.vstack([object_points, np.array([x, y, z])])

        # this is the same thing, but slower
        # object_points = np.vstack([object_points, np.linalg.inv(k).dot(z*np.array([u, v, 1])]))
        # np.inv is more computationally demanding than doing the algebra by hand 

    image1_points = np.delete(image1_points, delete, 0)
    image2_points = np.delete(image2_points, delete, 0)

    _, rvec, tvec, inliers = cv2.solvePnPRansac(object_points, image2_points, k, None)
    rmat = cv2.Rodrigues(rvec)[0]

    return rmat, tvec, image1_points, image2_points

In [ ]:
k, r, t, _, _, _, _ = cv2.decomposeProjectionMatrix(handler.P0)
k

In [ ]:
handler.reset_frames()
poses = (gt for gt in handler.gt)

In [ ]:
# np.printoptions(precision=4, suppress=True)
rmat, tvec, image1_points, image2_points = estimate_motion(matches, kp0, kp1, k, depth)
print('rotation matrix:')
print(rmat.round(4))
print('translation vector')
print(tvec.round(4))

In [ ]:
np.printoptions(precision=3, suppress=True)
transformation_matrix = np.hstack([rmat, tvec])
print(transformation_matrix.round(3))

In [ ]:
hom_trans_mat = np.eye(4)
hom_trans_mat

In [ ]:
hom_trans_mat[:3, :3] = rmat
hom_trans_mat[:3, 3] = tvec.T
hom_trans_mat.round(4)

In [ ]:
np.linalg.inv(hom_trans_mat)

In [ ]:
print(handler.gt[1].round(4))

In [ ]:
handler.reset_frames()
poses = (gt for gt in handler.gt)

In [ ]:
def visual_odometry(handler, detector='sift', matching='BF', filter_match_distance=None,
                    stereo_matcher='sgbm', mask=None, subset=None, plot=False):
    # determine if handler has lidar data
    lidar = handler.lidar

    # report methods being used to user
    print('generating disparities with stereo{}'.format(str.upper(stereo_matcher)))
    print('detecting features with {} and matching with {}'.format(str.upper(detector),
                                                                   matching))
    if filter_match_distance is not None:
        print('filtering matches with distance ratio of {}*distance'.format(filter_match_distance))
    if lidar:
        print('improving stereo depth estimation with lidar data')
    if subset is not None:
        num_frames = subset
    else:
        num_frames = handler.num_frames

    if plot:
        fig = plt.figure(figsize=(14, 14))
        ax = fig.add_subplot(projection='3d')
        ax.view_init(elev=-20, azim=270)
        xs = handler.gt[:, 0, 3]
        ys = handler.gt[:, 1, 3]
        zs = handler.gt[:, 2, 3]
        ax.set_box_aspect((np.ptp(xs), np.ptp(ys), np.ptp(zs)))
        ax.plot(xs, ys, zs, c='k')

    # establish a homogenous transformation matrix, first pose is identity
    T_tot = np.eye(4)
    trajectory = np.zeros((num_frames, 3, 4))
    trajectory[0] = T_tot[:3, :]
    imheight = handler.imheight
    imwidth = handler.imwidth

    # decompose left camera porjection matrix to get instrinsic k matrix
    k_left, r_left, t_left = decompose_projection_matrix(handler.P0)

    if handler.low_memory:
        handler.reset_frames()
        image_plus1 = next(handler.images_left)

    # iterate through all frames of the sequence
    for i in range(num_frames - 1):
        start = datetime.datetime.now()

        if handler.low_memory:
            image_left = image_plus1
            image_plus1 = next(handler.images_left)
            image_right = next(handler.images_right)
        else:
            image_left = handler.images_left[i]
            image_plus1 = handler.images_left[i+1]
            image_right = handler.images_right[i]

        depth = stereo_2_depth(image_left,
                               image_right,
                               P0=handler.P0,
                               P1=handler.P1,
                               matchers=stereo_matcher,
                               verbose=False)
        
        if lidar:
            if handler.low_memory:
                pointcloud = next(handler.pointclouds)
            else:
                pointcloud = handler.pointclouds[i]

            lidar_depth = pointcloud2image(pointcloud,
                                           imheight=imheight,
                                           imwidth=imwidth,
                                           Tr=handler.Tr,
                                           P0=handler.P0)

            indices = np.where(lidar_depth > 0)
            depth[indices] = lidar_depth[indices] # this is stereo depth, we substitute it with lidar depth

        # get keypoints and descriptors for left camera image of two sequential frames
        kp0, des0 = extract_features(image_left, detector, mask)
        kp1, des1 = extract_features(image_plus1, detector, mask)

        # get matches between features detected in two subsequent frames
        matches_unfilt = match_features(des0,
                                        des1,
                                        matching=matching,
                                        detector=detector)
        # print(f'number of matches before filtering: {len(matches_unfilt)}')

        # filter matches if a distance threshold is provided by user
        if filter_match_distance is not None:
            matches = filter_matches_distance(matches_unfilt, filter_match_distance)
        else:
            matches = matches_unfilt

        # print(f'number of matches after filtering: {len(matches)}')
        # print('length of kp0: {}, length of kp1: {}'.format(len(kp0), len(kp1)))

        # estimate motion between sequential images of the left camera
        # print(f"kp0: \n{kp0}")
        # print(f"kp1: \n{kp1}")
        # print(f"k_left: \n{k_left}")

        rmat, tvec, img1_points, img2_points = estimate_motion(matches,
                                                              kp0,
                                                              kp1,
                                                              k_left,
                                                              depth)

        # create a blank homogenous transformation matrix
        # print(f"rmat: {rmat.round(4)}")
        # print(f"tvec: {tvec.round(4)}")
        Tmat = np.eye(4)
        Tmat[:3, :3] = rmat
        Tmat[:3, 3] = tvec.T

        # print(f"Tmat: {Tmat.round(4)}")
        # print(f"T_tot: {T_tot.round(4)}")
        T_tot = T_tot @ np.linalg.inv(Tmat)
        # print(f"T_tot: {T_tot.round(4)}")

        trajectory[i+1, :, :] = T_tot[:3, :]

        end = datetime.datetime.now()
        print('time to compute frame {}:'.format(i+1), end-start)

        if plot:
            xs = trajectory[:i+2, 0, 3]
            ys = trajectory[:i+2, 1, 3]
            zs = trajectory[:i+2, 2, 3]
            plt.plot(xs, ys, zs, c='chartreuse')
            # plt.show()
            plt.pause(1e-32)

    # if plot:
        # plt.close()

    return trajectory

In [ ]:
trajectory_test = visual_odometry(handler,
                                  detector='sift',
                                  matching='BF',
                                  filter_match_distance=0.7,
                                  stereo_matcher='sgbm',
                                  mask=mask,
                                  # subset=50,
                                  plot=True)